In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [26]:
la_hispanic_2021 = pd.read_csv('https://raw.githubusercontent.com/jyyang7/reverse_mortgage/main/all_mortgage_data/la_hispanic_2021.csv', dtype={
    'county_code': str,
    'census_tract': str
})

In [27]:
rm_la_hispanic_2021 = la_hispanic_2021[(la_hispanic_2021['reverse_mortgage'] == 1)]
rm_la_hispanic_2021

,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open-end_line_of_credit,business_or_commercial_purpose,loan_amount,loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,multifamily_affordable_units,income,debt_to_income_ratio,applicant_credit_score_type,co-applicant_credit_score_type,applicant_ethnicity-1,applicant_ethnicity-2,applicant_ethnicity-3,applicant_ethnicity-4,applicant_ethnicity-5,co-applicant_ethnicity-1,co-applicant_ethnicity-2,co-applicant_ethnicity-3,co-applicant_ethnicity-4,co-applicant_ethnicity-5,applicant_ethnicity_observed,co-applicant_ethnicity_observed,applicant_race-1,applicant_race-2,applicant_race-3,applicant_race-4,applicant_race-5,co-applicant_race-1,co-applicant_race-2,co-applicant_race-3,co-applicant_race-4,co-applicant_race-5,applicant_race_observed,co-applicant_race_observed,applicant_sex,co-applicant_sex,applicant_sex_observed,co-applicant_sex_observed,applicant_age,co-applicant_age,applicant_age_above_62,co-applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus-1,aus-2,aus-3,aus-4,aus-5,denial_reason-1,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
408,2021,5493001FXF3J5C0QWQ87,31084,CA,06037,06037900102,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,White,Male,5,0,2,2,31,1,1,1,2,105000.0,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,2,2,2,2,NaN,1,1,3,5,1,NaN,31.0,NaN,9,9,1,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,2,4,5.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,2,4,1,5,2,4,55-64,9999,Yes,NaN,1,1,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,691,55.72,80000,38,155,487,51
563,2021,549300TINI6CL78UD591,31084,CA,06037,06037201302,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,White,Female,1,2,2,2,31,1,1,1,2,105000.0,NaN,2.83,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,675000,1,1,3,5,1,NaN,11.0,NaN,9,9,1,11.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,2,4,5.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,2,4,2,5,2,4,>74,9999,Yes,NaN,1,1,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,3652,78.59,80000,104,1161,484,32
564,2021,549300TINI6CL78UD591,31084,CA,06037,06037900501,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,White,Joint,4,0,2,2,31,1,1,1,2,105000.0,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,NaN,1,1,3,5,1,NaN,12.0,NaN,9,9,1,14.0,NaN,NaN,NaN,1.0,14.0,NaN,NaN,NaN,2,2,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,2,2,1,2,2,2,65-74,65-74,Yes,Yes,1,1,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,6429,78.16,80000,61,1046,2105,34
565,2021,549300TINI6CL78UD591,31084,CA,06037,06037901205,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,White,Joint,1,2,2,2,31,1,1,1,2,105000.0,NaN,2.66,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,385000,1,1,3,5,1,NaN,79.0,NaN,9,9,1,14.0,NaN,NaN,NaN,1.0,14.0,NaN,NaN,NaN,2,2,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,2,2,1,2,2,2,55-64,55-64,Yes,Yes,1,1,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,11642,44.89,80000,137,2536,3460,21
797,2021,2549001SI0EMFPT39035,31084,CA,06037,06037402903,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,White,Female,1,6,2,1,2,1,1,1,2,105000.0,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,1,NaN,1,1,3,5,1,NaN,2794.0,NaN,9,9,1,NaN,NaN,NaN,NaN,5.0,N

In [28]:
rm_la_hispanic_2021.to_csv('./reverse_mortgage_data/rm_la_hispanic_2021.csv', index=False)

In [3]:
la_all_2018_2021 = pd.read_csv('./all_mortgage_data/la_all_2018_2021.csv', dtype={
    'county_code': str,
    'census_tract': str
})

/var/folders/br/8x3cfw457y1gzxfpj7108n100000gn/T/ipykernel_32253/3467036358.py:1: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  la_all_2018_2021 = pd.read_csv('./all_mortgage_data/la_all_2018_2021.csv', dtype={


In [4]:
# filter to reverse mortgage

rm_la_all_2018_2021 = la_all_2018_2021[(la_all_2018_2021['reverse_mortgage'] == 1)]
rm_la_all_2018_2021

,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open-end_line_of_credit,business_or_commercial_purpose,loan_amount,loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,multifamily_affordable_units,income,debt_to_income_ratio,applicant_credit_score_type,co-applicant_credit_score_type,applicant_ethnicity-1,applicant_ethnicity-2,applicant_ethnicity-3,applicant_ethnicity-4,applicant_ethnicity-5,co-applicant_ethnicity-1,co-applicant_ethnicity-2,co-applicant_ethnicity-3,co-applicant_ethnicity-4,co-applicant_ethnicity-5,applicant_ethnicity_observed,co-applicant_ethnicity_observed,applicant_race-1,applicant_race-2,applicant_race-3,applicant_race-4,applicant_race-5,co-applicant_race-1,co-applicant_race-2,co-applicant_race-3,co-applicant_race-4,co-applicant_race-5,applicant_race_observed,co-applicant_race_observed,applicant_sex,co-applicant_sex,applicant_sex_observed,co-applicant_sex_observed,applicant_age,co-applicant_age,applicant_age_above_62,co-applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus-1,aus-2,aus-3,aus-4,aus-5,denial_reason-1,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
63750,2021,254900LTD432L5CHIZ48,31084,CA,06037,06037134720,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,White,Female,1,71,2,2,1,1,1,1,2,425000.0,NaN,3.07,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,2,765000,1,1,3,5,1,NaN,23.0,NaN,9,10,1.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,2,4,5.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,2,4,2,5,2,4,>74,9999,Yes,NaN,1,1,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,4859,74.54,80000,94,682,826,47
72163,2021,254900F9ZTVTX763V835,31084,CA,06037,06037460200,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Female,4,0,2,2,32,1,1,1,2,655000.0,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,NaN,1,1,3,5,1,NaN,18.0,NaN,9,9,2.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,2,4,5.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,2,4,2,5,2,4,>74,9999,Yes,NaN,1,1,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,5757,60.69,80000,166,1549,2108,67
72164,2021,254900F9ZTVTX763V835,31084,CA,06037,06037401601,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Black or African American,Joint,1,71,2,2,32,1,1,1,2,485000.0,58.55,3.32,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,825000.0,1,1,3,5,1,NaN,25.0,NaN,2,3,2.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2,2,3.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,2,2,1,2,2,2,>74,>74,Yes,Yes,1,1,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,5149,50.05,80000,124,1513,1881,39
72165,2021,254900F9ZTVTX763V835,31084,CA,06037,06037575103,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Male,1,71,2,2,32,1,1,1,2,385000.0,67.6,3.34,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,575000.0,1,1,3,5,2,NaN,21.0,NaN,2,10,2.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,2,4,5.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,2,4,1,5,2,4,>74,9999,Yes,NaN,1,1,6,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,4996,75.02,80000,63,334,742,49
72166,2021,254900F9ZTVTX763V835,31084,CA,06037,06037132101,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Asian,Female,1,71,2,2,32,1,1,1,2,545000.0,73.7,2.27,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,745000.

In [5]:
# export to csv

rm_la_all_2018_2021.to_csv('./reverse_mortgage_data/rm_la_all_2018_2021.csv', index=False)